# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
/Users/orin/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/orin/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/orin/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/U

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 152us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 41us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 42us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 35us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 37us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 37us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 37us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 37us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 38

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [6]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [7]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [9]:
# Train the model 
normalized_input_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 118us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 42us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 38us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 43us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 41us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 43us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 40us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 37us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 38us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 35us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 36us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 36

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [13]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                   y_train_scaled, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 158us/step - loss: 0.4368 - mean_squared_error: 0.4368 - val_loss: 0.1991 - val_mean_squared_error: 0.1991
Epoch 2/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.2275 - mean_squared_error: 0.2275 - val_loss: 0.1610 - val_mean_squared_error: 0.1610
Epoch 3/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.1842 - mean_squared_error: 0.1842 - val_loss: 0.1589 - val_mean_squared_error: 0.1589
Epoch 4/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.1642 - mean_squared_error: 0.1642 - val_loss: 0.1377 - val_mean_squared_error: 0.1377
Epoch 5/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.1480 - mean_squared_error: 0.1480 - val_loss: 0.1319 - val_mean_squared_error: 0.1319
Epoch 6/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.1329 - mean_squared_error: 0.1329 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0317 - mean_squared_error: 0.0317 - val_loss: 0.1013 - val_mean_squared_error: 0.1013
Epoch 51/150
1051/1051 [==============================] - 0s 35us/step - loss: 0.0315 - mean_squared_error: 0.0315 - val_loss: 0.1025 - val_mean_squared_error: 0.1025
Epoch 52/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0308 - mean_squared_error: 0.0308 - val_loss: 0.1028 - val_mean_squared_error: 0.1028
Epoch 53/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0301 - mean_squared_error: 0.0301 - val_loss: 0.1029 - val_mean_squared_error: 0.1029
Epoch 54/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0291 - mean_squared_error: 0.0291 - val_loss: 0.1043 - val_mean_squared_error: 0.1043
Epoch 55/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0286 - mean_squared_error: 0.0286 - val_loss: 0.1034 - val_mean_squared_error: 0.103

1051/1051 [==============================] - 0s 44us/step - loss: 0.0143 - mean_squared_error: 0.0143 - val_loss: 0.1112 - val_mean_squared_error: 0.1112
Epoch 100/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1104 - val_mean_squared_error: 0.1104
Epoch 101/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1118 - val_mean_squared_error: 0.1118
Epoch 102/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0137 - mean_squared_error: 0.0137 - val_loss: 0.1110 - val_mean_squared_error: 0.1110
Epoch 103/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.1113 - val_mean_squared_error: 0.1113
Epoch 104/150
1051/1051 [==============================] - 0s 35us/step - loss: 0.0133 - mean_squared_error: 0.0133 - val_loss: 0.1121 - val_mean_squared_error: 0.1121
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1164 - val_mean_squared_error: 0.1164
Epoch 149/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1163 - val_mean_squared_error: 0.1163
Epoch 150/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1176 - val_mean_squared_error: 0.1176


Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [14]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 28us/step


[0.007809019116303759, 0.007809019116303759]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [15]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 32us/step


[0.11757762089881607, 0.11757762089881607]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [16]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

26945.97132088095

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [19]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 168us/step - loss: 0.5083 - mean_squared_error: 0.5083 - val_loss: 0.1699 - val_mean_squared_error: 0.1699
Epoch 2/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.2218 - mean_squared_error: 0.2218 - val_loss: 0.1430 - val_mean_squared_error: 0.1430
Epoch 3/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.1795 - mean_squared_error: 0.1795 - val_loss: 0.1376 - val_mean_squared_error: 0.1376
Epoch 4/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.1573 - mean_squared_error: 0.1573 - val_loss: 0.1248 - val_mean_squared_error: 0.1248
Epoch 5/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.1410 - mean_squared_error: 0.1410 - val_loss: 0.1194 - val_mean_squared_error: 0.1194
Epoch 6/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.1265 - mean_squared_error: 0.1265 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0302 - mean_squared_error: 0.0302 - val_loss: 0.1090 - val_mean_squared_error: 0.1090
Epoch 51/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0296 - mean_squared_error: 0.0296 - val_loss: 0.1092 - val_mean_squared_error: 0.1092
Epoch 52/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0292 - mean_squared_error: 0.0292 - val_loss: 0.1103 - val_mean_squared_error: 0.1103
Epoch 53/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0286 - mean_squared_error: 0.0286 - val_loss: 0.1103 - val_mean_squared_error: 0.1103
Epoch 54/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0281 - mean_squared_error: 0.0281 - val_loss: 0.1106 - val_mean_squared_error: 0.1106
Epoch 55/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0278 - mean_squared_error: 0.0278 - val_loss: 0.1100 - val_mean_squared_error: 0.110

Epoch 99/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.1188 - val_mean_squared_error: 0.1188
Epoch 100/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0142 - mean_squared_error: 0.0142 - val_loss: 0.1184 - val_mean_squared_error: 0.1184
Epoch 101/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1184 - val_mean_squared_error: 0.1184
Epoch 102/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1178 - val_mean_squared_error: 0.1178
Epoch 103/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0138 - mean_squared_error: 0.0138 - val_loss: 0.1179 - val_mean_squared_error: 0.1179
Epoch 104/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.1203 - val_mean_squared_error: 

Epoch 148/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1241 - val_mean_squared_error: 0.1241
Epoch 149/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1237 - val_mean_squared_error: 0.1237
Epoch 150/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.1243 - val_mean_squared_error: 0.1243


Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 26us/step


[0.008237218075333523, 0.008237218075333523]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`) 

In [21]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 33us/step


[0.12434234934727956, 0.12434234934727956]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [23]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer='lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 186us/step - loss: 0.4769 - mean_squared_error: 0.4769 - val_loss: 0.1461 - val_mean_squared_error: 0.1461
Epoch 2/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.2096 - mean_squared_error: 0.2096 - val_loss: 0.1230 - val_mean_squared_error: 0.1230
Epoch 3/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.1698 - mean_squared_error: 0.1698 - val_loss: 0.1168 - val_mean_squared_error: 0.1168
Epoch 4/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.1479 - mean_squared_error: 0.1479 - val_loss: 0.1085 - val_mean_squared_error: 0.1085
Epoch 5/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.1331 - mean_squared_error: 0.1331 - val_loss: 0.1059 - val_mean_squared_error: 0.1059
Epoch 6/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.1197 - mean_squared_error: 0.1197 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0267 - mean_squared_error: 0.0267 - val_loss: 0.1159 - val_mean_squared_error: 0.1159
Epoch 51/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0262 - mean_squared_error: 0.0262 - val_loss: 0.1170 - val_mean_squared_error: 0.1170
Epoch 52/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0259 - mean_squared_error: 0.0259 - val_loss: 0.1189 - val_mean_squared_error: 0.1189
Epoch 53/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0253 - mean_squared_error: 0.0253 - val_loss: 0.1192 - val_mean_squared_error: 0.1192
Epoch 54/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0248 - mean_squared_error: 0.0248 - val_loss: 0.1188 - val_mean_squared_error: 0.1188
Epoch 55/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0244 - mean_squared_error: 0.0244 - val_loss: 0.1185 - val_mean_squared_error: 0.118

1051/1051 [==============================] - 0s 37us/step - loss: 0.0126 - mean_squared_error: 0.0126 - val_loss: 0.1289 - val_mean_squared_error: 0.1289
Epoch 100/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1282 - val_mean_squared_error: 0.1282
Epoch 101/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1289 - val_mean_squared_error: 0.1289
Epoch 102/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0123 - mean_squared_error: 0.0123 - val_loss: 0.1270 - val_mean_squared_error: 0.1270
Epoch 103/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0120 - mean_squared_error: 0.0120 - val_loss: 0.1278 - val_mean_squared_error: 0.1278
Epoch 104/150
1051/1051 [==============================] - 0s 37us/step - loss: 0.0119 - mean_squared_error: 0.0119 - val_loss: 0.1296 - val_mean_squared_error: 0.1296
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1348 - val_mean_squared_error: 0.1348
Epoch 149/150
1051/1051 [==============================] - 0s 36us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1331 - val_mean_squared_error: 0.1331
Epoch 150/150
1051/1051 [==============================] - 0s 35us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1344 - val_mean_squared_error: 0.1344


Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 27us/step


[0.007127974647585547, 0.007127974647585547]

Evaluate the model (`lecun_model`) on validate data (`X_val` and `y_val_scaled`) 

In [25]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 32us/step


[0.1343990405065365, 0.1343990405065365]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [29]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 296us/step - loss: 0.3205 - mean_squared_error: 0.3205 - val_loss: 0.1370 - val_mean_squared_error: 0.1370
Epoch 2/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.1666 - mean_squared_error: 0.1666 - val_loss: 0.1106 - val_mean_squared_error: 0.1106
Epoch 3/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.1242 - mean_squared_error: 0.1242 - val_loss: 0.1228 - val_mean_squared_error: 0.1228
Epoch 4/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0992 - mean_squared_error: 0.0992 - val_loss: 0.1091 - val_mean_squared_error: 0.1091
Epoch 5/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0888 - mean_squared_error: 0.0888 - val_loss: 0.1000 - val_mean_squared_error: 0.1000
Epoch 6/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0661 - mean_squared_error: 0.0661 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0090 - mean_squared_error: 0.0090 - val_loss: 0.1006 - val_mean_squared_error: 0.1006
Epoch 51/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0914 - val_mean_squared_error: 0.0914
Epoch 52/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.0863 - val_mean_squared_error: 0.0863
Epoch 53/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0089 - mean_squared_error: 0.0089 - val_loss: 0.0858 - val_mean_squared_error: 0.0858
Epoch 54/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0075 - mean_squared_error: 0.0075 - val_loss: 0.0915 - val_mean_squared_error: 0.0915
Epoch 55/150
1051/1051 [==============================] - 0s 40us/step - loss: 0.0070 - mean_squared_error: 0.0070 - val_loss: 0.0844 - val_mean_squared_error: 0.084

1051/1051 [==============================] - 0s 39us/step - loss: 0.0059 - mean_squared_error: 0.0059 - val_loss: 0.1102 - val_mean_squared_error: 0.1102
Epoch 100/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0048 - mean_squared_error: 0.0048 - val_loss: 0.0899 - val_mean_squared_error: 0.0899
Epoch 101/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.0989 - val_mean_squared_error: 0.0989
Epoch 102/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0045 - mean_squared_error: 0.0045 - val_loss: 0.0886 - val_mean_squared_error: 0.0886
Epoch 103/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.0927 - val_mean_squared_error: 0.0927
Epoch 104/150
1051/1051 [==============================] - 0s 39us/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0863 - val_mean_squared_error: 0.0863
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0967 - val_mean_squared_error: 0.0967
Epoch 149/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0917 - val_mean_squared_error: 0.0917
Epoch 150/150
1051/1051 [==============================] - 0s 38us/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0888 - val_mean_squared_error: 0.0888


Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [30]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 28us/step


[0.003577797645729542, 0.003577797645729542]

Evaluate the model (`rmsprop_model`) on val data (`X_val` and `y_val_scaled`) 

In [31]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 33us/step


[0.08884494258763673, 0.08884494258763673]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [32]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', loss='mse', metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 292us/step - loss: 0.3803 - mean_squared_error: 0.3803 - val_loss: 0.1651 - val_mean_squared_error: 0.1651
Epoch 2/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.1611 - mean_squared_error: 0.1611 - val_loss: 0.1334 - val_mean_squared_error: 0.1334
Epoch 3/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.1126 - mean_squared_error: 0.1126 - val_loss: 0.1044 - val_mean_squared_error: 0.1044
Epoch 4/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0864 - mean_squared_error: 0.0864 - val_loss: 0.1002 - val_mean_squared_error: 0.1002
Epoch 5/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0737 - mean_squared_error: 0.0737 - val_loss: 0.1004 - val_mean_squared_error: 0.1004
Epoch 6/150
1051/1051 [==============================] - 0s 44us/step - loss: 0.0563 - mean_squared_error: 0.0563 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0039 - mean_squared_error: 0.0039 - val_loss: 0.1073 - val_mean_squared_error: 0.1073
Epoch 51/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0038 - mean_squared_error: 0.0038 - val_loss: 0.0944 - val_mean_squared_error: 0.0944
Epoch 52/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.1182 - val_mean_squared_error: 0.1182
Epoch 53/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0053 - mean_squared_error: 0.0053 - val_loss: 0.0874 - val_mean_squared_error: 0.0874
Epoch 54/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0060 - mean_squared_error: 0.0060 - val_loss: 0.1033 - val_mean_squared_error: 0.1033
Epoch 55/150
1051/1051 [==============================] - 0s 42us/step - loss: 0.0050 - mean_squared_error: 0.0050 - val_loss: 0.0892 - val_mean_squared_error: 0.089

Epoch 99/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0868 - val_mean_squared_error: 0.0868
Epoch 100/150
1051/1051 [==============================] - 0s 41us/step - loss: 9.6988e-04 - mean_squared_error: 9.6988e-04 - val_loss: 0.0887 - val_mean_squared_error: 0.0887
Epoch 101/150
1051/1051 [==============================] - 0s 42us/step - loss: 6.6337e-04 - mean_squared_error: 6.6337e-04 - val_loss: 0.0881 - val_mean_squared_error: 0.0881
Epoch 102/150
1051/1051 [==============================] - 0s 44us/step - loss: 5.7098e-04 - mean_squared_error: 5.7098e-04 - val_loss: 0.0888 - val_mean_squared_error: 0.0888
Epoch 103/150
1051/1051 [==============================] - 0s 42us/step - loss: 3.9350e-04 - mean_squared_error: 3.9350e-04 - val_loss: 0.0872 - val_mean_squared_error: 0.0872
Epoch 104/150
1051/1051 [==============================] - 0s 44us/step - loss: 3.3683e-04 - mean_squared_error: 3.3683e-04 - val

1051/1051 [==============================] - 0s 42us/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0834 - val_mean_squared_error: 0.0834
Epoch 148/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0887 - val_mean_squared_error: 0.0887
Epoch 149/150
1051/1051 [==============================] - 0s 41us/step - loss: 0.0025 - mean_squared_error: 0.0025 - val_loss: 0.0816 - val_mean_squared_error: 0.0816
Epoch 150/150
1051/1051 [==============================] - 0s 43us/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0874 - val_mean_squared_error: 0.0874


Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [33]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 29us/step


[0.0016768582822749662, 0.0016768582822749662]

Evaluate the model (`adam_model`) on validation data (`X_val` and `y_val_scaled`) 

In [34]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 36us/step


[0.08737496911289575, 0.08737496911289575]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [35]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 34us/step


[0.16383574923423871, 0.16383574923423871]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [36]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

31807.961380040837

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.